# Workshop 2: Connectivity principles and structure function analysis

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


In this workshop, we explore different two principles that govern which neurons are connected with each other:

1. Spatial distance between neurons
2. Correlation of acitivity traces during specific stimuli

In [ ]:
mat_version = 1169

import os
import sys
from os.path import join as pjoin

HOME = os.path.expanduser("~")

# NOTE: adjust for your system
# data_dir = f"{HOME}/SWDB_2025_Connectomics/data/"
# utils_dir = f"{HOME}/SWDB_2025_Connectomics/code/utils"

# # NOTE: adjust for your system
data_dir = pjoin('..','..','data', f"v1dd_{mat_version}")
utils_dir = pjoin("..","utils")

sys.path.append(utils_dir)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<h2> Import packages </h2>

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import itertools
from typing import Union, Optional
from scipy import stats, spatial
import itertools
import tqdm
from os.path import join as pjoin
import collections

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Preparation

First, we are loading data files that were introduced in the first workshop such as the synapse and cell tables, and information about proofreading. This workshop does not require the CAVEclient.

In [ ]:
# Move this to utils

def filter_synapse_table(
    synapse_table: pd.DataFrame, pre_root_ids=None, post_root_ids=None
):
    """Filter synapse table by pre and post root ids.

    Args:
        synapse_table: synapse table with pre_pt_root_ids and post_pt_root_ids as pd.DataFrame
        pre_root_ids: np.ndarray, list or pd.Series if root_ids to filter on the presynaptic side
        post_root_ids: np.ndarray, list or pd.Series if root_ids to filter on the postsynaptic side

    Returns:
        synapse_table: filtered synapse table
    """

    if pre_root_ids is not None:
        assert isinstance(pre_root_ids, (np.ndarray, list, pd.core.series.Series)), (
            f"IDs have to be of type np.ndarray, list or pd.Series; got {type(pre_root_ids)}"
        )
        pre_mask = np.isin(synapse_table["pre_pt_root_id"], pre_root_ids)
    else:
        pre_mask = np.ones(len(synapse_table), dtype=bool)

    if post_root_ids is not None:
        assert isinstance(post_root_ids, (np.ndarray, list, pd.core.series.Series)), (
            f"IDs have to be of type np.ndarray, list or pd.Series; got {type(pre_root_ids)}"
        )
        post_mask = np.isin(synapse_table["post_pt_root_id"], post_root_ids)
    else:
        post_mask = np.ones(len(synapse_table), dtype=bool)

    return synapse_table[pre_mask & post_mask]


Loading proofreading, cell and synapse information.

In [ ]:
# Loads cells with axon and dendrite proofreading
dendrite_proof_root_ids = np.load(pjoin(data_dir, f'proofread_dendrite_list_{mat_version}.npy'))
axon_proof_root_ids = np.load(pjoin(data_dir, f'proofread_axon_list_{mat_version}.npy'))

print(f'There are {len(dendrite_proof_root_ids)} cells with acceptable dendrites, and {len(axon_proof_root_ids)} cells with axon proofreading')
print(f'The number of cells with usable axons and dendrites is: {len(np.isin(axon_proof_root_ids, dendrite_proof_root_ids))}')

In [ ]:
syn_df = pd.read_feather(f"{data_dir}/syn_df_all_to_proofread_to_all_{mat_version}.feather")
cell_df = pd.read_feather(f"{data_dir}/soma_and_cell_type_{mat_version}.feather")

In [ ]:
cell_df["cell_type"].value_counts()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## #1: Connection probability vs distance

Neurons are distributed in space. Lets look at the dataset from above. For now, we select one of the cell types available to us: L3-IT

In [ ]:
sub_cell_df = cell_df[cell_df["cell_type"] == "L3-IT"]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

The dataset was not imaged in a perfectly upright position. We corrected for that and created the `*trform*` columns for each neuron position. We will use these for our analysis. However, to find neurons in neuroglancer, one has to still use the not-transformed positions.

In [ ]:
sub_cell_df.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(sub_cell_df["pt_position_trform_x"] / 1000., sub_cell_df["pt_position_trform_z"] / 1000., c="k", s=1)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

To test whether distance and connectedness are correlated, we calculate the _lateral_ distance between pairs of neurons. The _lateral_ distance ignores cortical depth. For a pair of neurons the lateral distance can be calculated as the norm of the difference of their coordinates:

In [ ]:
pos_cols = ["pt_position_trform_x", "pt_position_trform_z"]

difference = sub_cell_df.iloc[0][pos_cols] - sub_cell_df.iloc[1][pos_cols]
np.linalg.norm(difference)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

This function computes the lateral distances across all neurons efficiently.

In [ ]:
def calculate_lateral_distances(cell_df):
    """ Calculates the lateral distances in µm between all neurons in the dataframe."""
    
    lateral_locations = np.array(cell_df[["pt_position_trform_x", "pt_position_trform_z"]])
    lateral_distances = spatial.distance.cdist(lateral_locations, lateral_locations) / 1000.
    
    id_pairs = list(itertools.product(cell_df["pt_root_id"], cell_df["pt_root_id"]))
    
    lateral_distance_df = pd.DataFrame(id_pairs, columns=['pre_pt_root_id', 'post_pt_root_id'])
    lateral_distance_df['distance'] = lateral_distances.flatten()

    lateral_distance_df = lateral_distance_df[lateral_distance_df['pre_pt_root_id'] != lateral_distance_df['post_pt_root_id']]
    return lateral_distance_df

In [ ]:
lateral_distance_df = calculate_lateral_distances(sub_cell_df)
lateral_distance_df.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

sns.histplot(lateral_distance_df, x="distance", fill=True, alpha=.5, color="k", lw=3, element="step")

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  

ax.spines['top'].set_visible(False) 
ax.spines['right'].set_visible(False) 
ax.spines['bottom'].set_linewidth(2)  
ax.spines['left'].set_linewidth(2) 

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Count", fontsize=15)

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

This distribution shows the distance between all pairs of L3-IT neurons in the dataset. Next, we want to compare this distribution with the distances between the all _connected_ neuron pairs to see if they differ. For that, we combine the IDs we selected from our cell table with the proofreading information. Let's gather the proofread L3-IT neurons and use them to filter the synapse table.

In [ ]:
pre_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], axon_proof_root_ids)]
post_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], dendrite_proof_root_ids)]

analysis_syn_df = filter_synapse_table(syn_df, pre_root_ids, post_root_ids).reset_index(drop=True)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Some neurons are connected with multiple synapses. Here, we will add them together to a single connection.

</div>

In [ ]:
conn_df = analysis_syn_df.groupby(["pre_pt_root_id", "post_pt_root_id"])["size"].sum().reset_index()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Finally, we need to combine distance and connectivity information. We have this information in two separate dataframes:

In [ ]:
conn_df.head(5)

In [ ]:
lateral_distance_df.head(5)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> For each connection, we would like to know what the distance is between the two neurons. Write the merge function to combine the distance and synapse information of the two tables. Note that the connectivity table has far fewer entries than the distance table. By merging them, we can filter the distance table down to the neuron pairs for which we have a synaptic connection.

</div>

In [ ]:
# conn_dist_df = 

In [ ]:
conn_dist_df = pd.merge(conn_df, lateral_distance_df, on=["pre_pt_root_id", "post_pt_root_id"])

In [ ]:
conn_dist_df.shape, conn_df.shape, lateral_distance_df.shape

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Let's compare the two distribution in plot. We use `stat="probability"` to plot them at the same scale.

</d>

In [ ]:
bins = np.linspace(0, 1000, 101)

fig, ax = plt.subplots(figsize=(8, 5))

sns.histplot(lateral_distance_df, x="distance", stat="probability", bins=bins, element="step", label="all pairs", lw=3, fill=True, alpha=.5, color="k")
sns.histplot(conn_dist_df, x="distance", stat="probability", bins=bins, element="step", label="connected pairs", lw=3, fill=True, alpha=.5, color="r")

ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)

sns.despine(offset=15, trim=False)
plt.tight_layout()

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

To get a better intution, we will look at some examples of connected neuron pairs across distance:

- 10µm: Neuroglancer link
- 50µm: Neuroglancer link
- 100µm: Neuroglancer link
- 500µm: Neuroglancer link

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Question:</b> Is this a fair comparison? Does our NULL distribution, the one of all pairs, represent the set of possible connections well?

</div>

In [ ]:
lateral_distance_proofread_df = lateral_distance_df.copy()
lateral_distance_proofread_df = lateral_distance_proofread_df[np.isin(lateral_distance_proofread_df["pre_pt_root_id"], pre_root_ids)]
lateral_distance_proofread_df = lateral_distance_proofread_df[np.isin(lateral_distance_proofread_df["post_pt_root_id"], post_root_ids)]

In [ ]:
bins = np.linspace(0, 1000, 101)

fig, ax = plt.subplots(figsize=(8, 5))

sns.histplot(lateral_distance_proofread_df, x="distance", stat="probability", bins=bins, element="step", label="proofread pairs", lw=3, fill=True, alpha=.5, color="k")
sns.histplot(conn_dist_df, x="distance", stat="probability", bins=bins, element="step", label="connected pairs", lw=3, fill=True, alpha=.5, color="r")

ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)
sns.despine(offset=15, trim=False)
plt.tight_layout()

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Another way to represent the data above is to calculate a "connection probability". That is, <b>how likely is it that two neurons are connected given their lateral distance from each other? </b> 

There are different ways to achieve this, here we will calculate the connection probability for every distance bin.

In [ ]:
bins = np.linspace(0, 500, 101)
proofread_pair_counts = np.histogram(lateral_distance_proofread_df["distance"], bins)[0]
connected_pair_counts = np.histogram(conn_dist_df["distance"], bins)[0]

connection_probability = connected_pair_counts / proofread_pair_counts

connection_probability_df = pd.DataFrame({"bin_id": np.arange(len(bins)-1), "probability": connection_probability})

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

sns.lineplot(x=bins[:-1], y=connection_probability, lw=3, color="k")

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Lateral distance [µm]", fontsize=15)
ax.set_ylabel("Connection probability", fontsize=15)

sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## #2: Structure vs function: Connectivity vs activity correlation

The V1dd dataset is a structure function dataset. We have functional measurements for many cells in the datasets. These are very rich and can be related to behavioral data. The data is stored in nwb files. Here is an example of the data that is accesible:

In [ ]:
# Plot of nwb files

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Activity correlations</b> For the purpose of this workshop, we precalculated correlations between cells. To be more precise, we calculated the total correlation of the activity traces of neurons imaged during the same session during the same stimulus condition.

If you want to use the V1dd dataset for a project, this kind of analysis could be extended to include behavioral data and create more sophisticated functional comparisons between cells which can then be related to the connectivity data.

<b>Coregistration</b>: To analyze the functional and connectivity data, neurons in both datasets need to be _coregistered_, that is an ROI in the activity stack needs to be linked with a cell body in the EM stack. The data below only contains those neurons which were coregistered.

In [ ]:
corr_coreg_df = pd.read_feather(f"{data_dir}/cell_cell_correlations_by_stimulus_coregistered.feather")

In [ ]:
corr_coreg_df.head(5)

In [ ]:
stimulus_conditions = corr_coreg_df.columns[3:]
stimulus_conditions

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Can we treat the different stimulus conditions as separate experiments or would we expect all results to be similar no matter what stimulus condition we base or analysis on? Next, we analyze how correlated the co-activities of cells are between the different stimulus conditions. For that, we calculate the correlation of the pairwise correlations. 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Discussion:</b> What outcome do you expect and what do different results mean for the use of the neuron-neuron correlations for further analysis?

</div>

In [ ]:
stim_stim_corr_df = corr_coreg_df[stimulus_conditions].corr()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=200)

heatmap = sns.heatmap(stim_stim_corr_df, ax=ax, vmax=.5, vmin=0)
colorbar = heatmap.collections[0].colorbar
colorbar.set_label("Correlation of neuron-neuron corrs.", fontsize=15)

ax.tick_params(axis='both', which='major', labelsize=16, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xticklabels([])
ax.set_xlabel("")
ax.set_ylabel("")
sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Another way to compare the different stimulus conditions is to plot the distributions of the neuron-neuron correlations.

In [ ]:
stim = stimulus_conditions[2]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=200)

sns.histplot(data=corr_coreg_df, x=stim, label=stim, bins=np.linspace(-1, 1, 201), 
             element="step", lw=3, ax=ax, stat="probability", alpha=.5)

ax.legend(frameon=False)
ax.tick_params(axis='both', which='major', labelsize=16, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine
ax.set_xlim(-.1, .3)

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron-pairs", fontsize=16)
sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> Add the distributions for the other stimulus conditions into the same plot.

</div>

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=200)

for stim in stimulus_conditions:
    sns.histplot(data=corr_coreg_df, x=stim, label=stim, bins=np.linspace(-1, 1, 201), 
                 element="step", lw=3, ax=ax, stat="probability", alpha=.5)

ax.legend(frameon=False)
ax.tick_params(axis='both', which='major', labelsize=16, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine
ax.set_xlim(-.1, .3)

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron-pairs", fontsize=16)
sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

With the activity correlation between neurons in hand, we want to answer one question: <b>Are neurons that are more correlated in their activity, more likely to be connected with each other?</b>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

As before, we will limit this analysis to L3-IT neurons. We collect the connectivity information between these neurons as before.

In [ ]:
sub_cell_df = cell_df[cell_df["cell_type"] == "L3-IT"]

In [ ]:
pre_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], axon_proof_root_ids)]
post_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], dendrite_proof_root_ids)]

In [ ]:
analysis_syn_df = filter_synapse_table(syn_df, axon_proof_root_ids, dendrite_proof_root_ids).reset_index(drop=True)
conn_df = analysis_syn_df.groupby(["pre_pt_root_id", "post_pt_root_id"])["size"].sum().reset_index()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

We will limit our analysis to just those neurons for which we have proofreading data available

In [ ]:
corr_coreg_proofread_df = corr_coreg_df.copy()
corr_coreg_proofread_df = corr_coreg_proofread_df[np.isin(corr_coreg_proofread_df["pre_pt_root_id"], pre_root_ids)]
corr_coreg_proofread_df = corr_coreg_proofread_df[np.isin(corr_coreg_proofread_df["post_pt_root_id"], post_root_ids)]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Let's compare the connectivity and the activity correlation data.

In [ ]:
conn_corr_df = pd.merge(conn_df, corr_coreg_proofread_df, on=["pre_pt_root_id", "post_pt_root_id"])

In [ ]:
corr_coreg_proofread_df.shape, conn_corr_df.shape

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

For a given stimulus, we compare the distributions of activity correlations between the connected pairs with the activity correlations of all pairs.

In [ ]:
stim = stimulus_conditions[2]
stim

In [ ]:
bins=np.linspace(-.1, .4, 26)

fig, ax = plt.subplots(figsize=(8, 5))

ax.set_title(stim, fontsize=18)

sns.histplot(corr_coreg_proofread_df, x=stim, stat="probability", bins=bins, element="step", label="proofread pairs", lw=3, fill=True, alpha=.5, color="k")
sns.histplot(conn_corr_df, x=stim, stat="probability", bins=bins, element="step", label="connected pairs", lw=3, fill=True, alpha=.5, color="r")

ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)
sns.despine(offset=15, trim=False)
plt.tight_layout()

plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Just like for the distance analysis above (#1), we can express this result as a "connection probability" for a given correlation. That is, how likely is it that two neurons are connected given their activity correlation? 

Here, we will calculate the connection probability bins of activity correlations.

In [ ]:
bins = np.percentile(corr_coreg_proofread_df[stim], np.linspace(0, 100, 6)) # Makes 5 bins

bin_centers = np.convolve(bins, [.5, .5], mode="valid") # Calulates the center of each bin

proofread_pair_counts = np.histogram(corr_coreg_proofread_df[stim], bins)[0]
connected_pair_counts = np.histogram(conn_corr_df[stim], bins)[0]

connection_probability = connected_pair_counts / (proofread_pair_counts + 1e-6)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))

sns.lineplot(x=bin_centers, y=connection_probability, lw=3, ls="--", legend=False, ax=ax, marker="o", markersize=15)

ax.legend(frameon=False, fontsize=14)
ax.get_legend().set_bbox_to_anchor([1.1, 1])
ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Connection probability", fontsize=15)

sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> Add the distributions for the other stimulus conditions into the same plot.

</div>

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))

for stim in stimulus_conditions:
    bins = np.percentile(corr_coreg_proofread_df[stim], np.linspace(0, 100, 6))#[[0, 1, -2, -1]]

    bin_centers = np.convolve(bins, [.5, .5], mode="valid")#[[0, -1]]
    proofread_pair_counts = np.histogram(corr_coreg_proofread_df[stim], bins)[0]#[[0, -1]]
    connected_pair_counts = np.histogram(conn_corr_df[stim], bins)[0]#[[0, -1]]
    
    connection_probability = connected_pair_counts / (proofread_pair_counts + 1e-6)
    
    sns.lineplot(x=bin_centers, y=connection_probability, lw=3, ls="--", label=stim, legend=False, ax=ax, marker="o", markersize=15)

ax.legend(frameon=False, fontsize=14)
ax.get_legend().set_bbox_to_anchor([1.1, 1])
ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Connection probability", fontsize=15)

sns.despine(offset=15, trim=False)
plt.tight_layout()
plt.show()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

### Summary

We showed that for L3-IT cells

1. neurons that are closer to each other, are more likely to be connected.
2. neurons with more correlated activity traces, are more likely to be connected.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## A stronger NULL model: Can distance $\rightarrow$ connectivity account for structure-function correlation

So far we implicity used a _NULL model_ where all neuron pairs have the same likelihood to be connected. But we just showed that spatial separation alone increases the probability that two neurons are connected. Can this account for the correlation between activity correlation and connectivity?

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Collecting data products:</b> Let's port the results from our previous analyses

In [ ]:
sub_cell_df = cell_df[cell_df["cell_type"] == "L3-IT"]
distance_bins = np.linspace(0, 500, 101)

In [ ]:
# Connectivity

pre_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], axon_proof_root_ids)]
post_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], dendrite_proof_root_ids)]

analysis_syn_df = filter_synapse_table(syn_df, axon_proof_root_ids, dendrite_proof_root_ids).reset_index(drop=True)
conn_df = analysis_syn_df.groupby(["pre_pt_root_id", "post_pt_root_id"])["size"].sum().reset_index()

In [ ]:
# Distance 

lateral_distance_df = calculate_lateral_distances(sub_cell_df)

pre_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], axon_proof_root_ids)]
post_root_ids = sub_cell_df["pt_root_id"][np.isin(sub_cell_df["pt_root_id"], dendrite_proof_root_ids)]

analysis_syn_df = filter_synapse_table(syn_df, pre_root_ids, post_root_ids).reset_index(drop=True)
conn_df = analysis_syn_df.groupby(["pre_pt_root_id", "post_pt_root_id"])["size"].sum().reset_index()

lateral_distance_proofread_df = lateral_distance_df.copy()
lateral_distance_proofread_df = lateral_distance_proofread_df[np.isin(lateral_distance_proofread_df["pre_pt_root_id"], pre_root_ids)]
lateral_distance_proofread_df = lateral_distance_proofread_df[np.isin(lateral_distance_proofread_df["post_pt_root_id"], post_root_ids)]

In [ ]:
# Distance vs Connectivity

conn_dist_df = pd.merge(conn_df, lateral_distance_df, on=["pre_pt_root_id", "post_pt_root_id"])

proofread_pair_counts = np.histogram(lateral_distance_proofread_df["distance"], distance_bins)[0]
connected_pair_counts = np.histogram(conn_dist_df["distance"], distance_bins)[0]

connection_probability = connected_pair_counts / proofread_pair_counts
connection_probability_df = pd.DataFrame({"bin_id": np.arange(len(distance_bins)-1), "probability": connection_probability}).set_index("bin_id")

In [ ]:
# Activity correlation

corr_coreg_proofread_df = corr_coreg_df.copy()
corr_coreg_proofread_df = corr_coreg_proofread_df[np.isin(corr_coreg_proofread_df["pre_pt_root_id"], pre_root_ids)]
corr_coreg_proofread_df = corr_coreg_proofread_df[np.isin(corr_coreg_proofread_df["post_pt_root_id"], post_root_ids)]

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Let's combine our main free dataframes into one

In [ ]:
conn_df.head(5)

In [ ]:
lateral_distance_proofread_df.head(5)

In [ ]:
corr_coreg_proofread_df.head(5)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> Merge the three dataframes into one. Make sure not to loose any rows in the merging by using `how="left"` or `how="right"`. This is particularly important for the connectivity table. We can set the synapse size of the neuron pairs which are not connected to 0. 

</div>

In [ ]:
# full_df = ...

In [ ]:
full_df = corr_coreg_proofread_df.copy()
full_df = pd.merge(full_df, lateral_distance_proofread_df, on=["pre_pt_root_id", "post_pt_root_id"])
full_df = pd.merge(full_df, conn_df, on=["pre_pt_root_id", "post_pt_root_id"], how="left").fillna(0)

In [ ]:
full_df["connected"] = np.array(full_df["size"] > 0)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Measurement:</b> We will summarize the structure-function result with one value, the mean correlation of the cells in the connected and overall group.

In [ ]:
full_df.groupby("connected")[stimulus_conditions].mean()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Distance-based null model to sample connectomes:</b> Instead of using the not-connected neurons as our NULL model for comparison, we want to use our insight about the connection probabilities vs neuron distance to create a stronger NULL model. We will achieve this by sampling new "connectomes" based on the distance-based connection probabilities. For each new connectome, we will calculate the correlation of the connected (sampled) neurons.

First, we assign probabilities to each potential connection:

In [ ]:
connection_probability_df.head(5)

In [ ]:
assigned_distance_bins = np.digitize(full_df["distance"], distance_bins)
full_df["connection_probability"] = np.array(connection_probability_df.loc[assigned_distance_bins])

In [ ]:
full_df.head(5)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<b>Sampling connectomes:</b> We sample connectomes using `np.random.choice`. This function takes probabilities which have to sum to one as input as well as a number of samples that should be taken.

In [ ]:
full_df["connection_probability_norm"] = full_df["connection_probability"] / full_df["connection_probability"].sum()

In [ ]:
#Single run:

full_sampled_df = full_df.copy()
sampled_idx = np.random.choice(full_sampled_df.index, p=full_df["connection_probability_norm"], size=np.sum(full_sampled_df["connected"]))
full_sampled_df["connected_sampled"] = False
full_sampled_df.loc[sampled_idx, "connected_sampled"] = True

In [ ]:
full_sampled_df.loc[full_sampled_df["connected_sampled"], stimulus_conditions].mean()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

We repeat the sampling 10,000 times and combine the result in a DataFrame.

In [ ]:
sampled_means = []

for _ in tqdm.tqdm(range(10_000)):
    sampled_idx = np.random.choice(
        full_sampled_df.index,
        p=full_df["connection_probability_norm"],
        size=np.sum(full_sampled_df["connected"])
    )
    
    full_sampled_df["connected_sampled"] = False
    full_sampled_df.loc[sampled_idx, "connected_sampled"] = True
    
    means = full_sampled_df.loc[full_sampled_df["connected_sampled"], stimulus_conditions].mean()
    sampled_means.append(means)

# Combine into a DataFrame (each row = one sample)
sampled_means_df = pd.DataFrame(sampled_means)

In [ ]:
sampled_means_df.head(5)

In [ ]:
sampled_means_df.mean()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

With the results in hand, lets plot our distribution for a single stimulus condition.

In [ ]:
stim = stimulus_conditions[2]
stim

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.set_title(stim, fontsize=18)

sns.histplot(data=sampled_means_df, x=stim, stat="percent", bins=101, element="step", label="Distance-based\nconnected samples", 
             lw=3, fill=True, alpha=.5, color="k")

ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)
sns.despine(offset=15, trim=False)
plt.tight_layout()

plt.show()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<p><b>Task:</b> How different is our distribution from the observed correlation of the connected neurons? Calculate the zscore between the observed result and the measured correlation of the connected neurons.

</div>

In [ ]:
(full_df[full_df["connected"]][stimulus_conditions].mean() - sampled_means_df.mean()) / sampled_means_df.std()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

Visualizing it all in one plot:

In [ ]:
stim = stimulus_conditions[2]
stim

In [ ]:
zscore = (full_df[full_df["connected"]][stim].mean() - sampled_means_df[stim].mean()) / sampled_means_df[stim].std()

fig, ax = plt.subplots(figsize=(8, 5))

ax.set_title(stim, fontsize=18)

sns.histplot(data=sampled_means_df, x=stim, stat="percent", bins=101, element="step", label="Distance-based\nconnected samples", 
             lw=3, fill=True, alpha=.5, color="k")

ax.axvline(x=full_df[full_df["connected"]][stim].mean(), color='r', linestyle='--', label='Observed:\nConnected', lw=3)
ax.axvline(x=full_df[~full_df["connected"]][stim].mean(), color='b', linestyle='--', label='Observed:\nNot connected', lw=3)

ax.text(ax.get_xlim()[0], ax.get_ylim()[1] * .95, f"Z-Score: {zscore:.2f}", fontsize=14)

ax.legend(frameon=False, fontsize=14)

ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=8)  # Set larger tick labels
ax.tick_params(axis='both', which='minor', labelsize=14, width=1, length=4)  # Set larger tick labels

ax.spines['top'].set_visible(False)  # Hide the top spine
ax.spines['right'].set_visible(False)  # Hide the right spine
ax.spines['bottom'].set_linewidth(2)  # Hide the top spine
ax.spines['left'].set_linewidth(2)  # Hide the right spine

ax.set_xlabel("Total trace correlation during stimulus", fontsize=16)
ax.set_ylabel("Fraction of neuron pairs", fontsize=15)
sns.despine(offset=15, trim=False)
plt.tight_layout()

plt.show()